# Web scraping 

Import libraries

In [1]:
import pandas as pd
import time
import json
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [2]:
# Create service
webdriver_service = Service(ChromeDriverManager().install())
# Create driver
driver = webdriver.Chrome(service = webdriver_service)

In [3]:
# Go to the the main page of the site
page_url = "https://olympics.com/en/olympic-games"
driver.get(page_url)

In [4]:
# # Click on Accept cookies
#time.sleep(3)
#driver.find_element(By.XPATH, '//button[text()="Yes, I am happy"]').click()

In [5]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(5)

In [6]:
games = json.loads(driver.find_element(By.CSS_SELECTOR, 'script[type="application/ld+json"]').get_attribute("innerHTML"))

Getting links to pages with olympic games data

In [7]:
result_link_list = []
for game in games['itemListElement']:
    link = game['url'].split('olympic-games')[0] + 'en/olympic-games' + game['url'].split('olympic-games')[1] + '/results'
    link_temp = ''
    driver.get(link)
    try:
        discipline = json.loads(driver.find_element(By.CSS_SELECTOR, 'script[type="application/json"]').get_attribute("innerHTML"))
    except:
        continue
    for disc in discipline['props']['pageProps']['olympicGame']['disciplines']:
        if 'rhythmic' in disc['slug']:
            link_temp = link + '/' + disc['slug']
            break
    if link_temp != '':
        driver.get(link_temp)
        result_link = driver.find_element(By.CSS_SELECTOR, 'div[data-cy*="full-results-link"] a[href*="individual"]').get_attribute("href")
        if len(result_link) > 1:
            result_link_list.append(result_link)
result_link_list

['https://olympics.com/en/olympic-games/tokyo-2020/results/rhythmic-gymnastics/individual-all-around',
 'https://olympics.com/en/olympic-games/rio-2016/results/gymnastics-rhythmic/individual-all-around-women',
 'https://olympics.com/en/olympic-games/london-2012/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/en/olympic-games/beijing-2008/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/en/olympic-games/athens-2004/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/en/olympic-games/sydney-2000/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/en/olympic-games/atlanta-1996/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/en/olympic-games/barcelona-1992/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/en/olympic-games/seoul-1988/results/gymnastics-rhythmic/individual-all-round-women',
 'https://olympics.com/

Getting data 

In [8]:
game_l = []
gym_place_l = []
gym_country_l = []
gym_name_l = []
gym_score_l = []
for page in result_link_list:
    driver.get(page)
    name_game = driver.find_element(By.CSS_SELECTOR, 'h1[class*="styles__Header"] span').text
    gym_place = driver.find_elements(By.CSS_SELECTOR, 'div[data-cy="table-content"] div[data-cy*="medal-row"]')
    gym_country = driver.find_elements(By.CSS_SELECTOR, 'div[data-cy="table-content"] span[class*="CountryName"]')
    gym_name = driver.find_elements(By.CSS_SELECTOR, 'div[data-cy="table-content"] h3[data-cy="athlete-name"]')
    gym_score = driver.find_elements(By.CSS_SELECTOR, 'div[data-cy="table-content"] span[data-cy="result-info-content"]')
    for index in range(10):
        game_l.append(name_game)
        gym_place_l.append(int(gym_place[index].get_attribute("data-cy").split('-')[-1]))
        if gym_country[index].text=='ROC':
            gym_country_l.append('RUS')
        elif gym_country[index].text=='EUN':
            gym_country_l.append('UKR')
        else:
            gym_country_l.append(gym_country[index].text)
        gym_name_l.append(gym_name[index].text)
        gym_score_l.append(gym_score[index].text)

In [9]:
data = pd.DataFrame({"game" : game_l, "place" : gym_place_l, "country" : gym_country_l, "name" : gym_name_l, "score" : gym_score_l})
data

,game,place,country,name,score
0,TOKYO 2020,1,ISR,Linoy Ashram,107.800
1,TOKYO 2020,2,RUS,Dina Averina,107.650
2,TOKYO 2020,3,BLR,Alina Harnasko,102.700
3,TOKYO 2020,4,RUS,Arina Averina,102.100
4,TOKYO 2020,5,BUL,Boryana Kaleyn,100.625
...,...,...,...,...,...
95,LOS ANGELES 1984,6,ESP,Marta Canton Gutierrez,56.950
96,LOS ANGELES 1984,7,ITA,Giulia Staccioli,56.775
97,LOS ANGELES 1984,8,JPN,Hiroko Yamasaki,56.675
98,LOS ANGELES 1984,9,ESP,Marta Bobo Arce,56.375


Saving dataset in csv file

In [10]:
data.to_csv('data_rg.csv', index=False)